In [22]:
import numpy as np
import pandas as pd
import h5py
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
    
from pyccapt.calibration.data_tools import data_loadcrop

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
filename = 'D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4.h5'
df_dld = data_loadcrop.fetch_dataset_from_dld_grp(filename, extract_mode='dld')

{'apt': ['high_voltage', 'main_chamber_vacuum', 'num_events', 'pulse', 'temperature', 'time_counter'], 'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'pulse', 'start_counter', 'time_data'], 'time': ['time_h', 'time_m', 'time_s']}


In [30]:
df_dld

,high_voltage (V),pulse,start_counter,t (ns),x_det (cm),y_det (cm)
0,600.000000,328.0,8202,2537.802979,1.080816,0.006531
1,615.000000,328.0,14741,3686.929443,1.443265,-1.812245
2,624.979980,328.0,2657,3110.466553,-0.688980,-2.249796
3,624.979980,328.0,4568,1171.380737,0.192653,-0.914286
4,634.919983,328.0,4498,2703.307129,0.058776,1.479184
...,...,...,...,...,...,...
12312991,8000.000000,1600.0,11089,3722.090332,2.282449,2.798367
12312992,8000.000000,1600.0,13935,3065.292725,3.725714,-0.675918
12312993,8000.000000,1600.0,2722,2561.627686,3.229388,1.573878
12312994,8000.000000,1600.0,3387,3579.656494,0.414694,2.693877


In [31]:
filename = 'D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4.h5'
df_tdc = data_loadcrop.fetch_dataset_from_dld_grp(filename, extract_mode='tdc_sc')

{'apt': ['high_voltage', 'main_chamber_vacuum', 'num_events', 'pulse', 'temperature', 'time_counter'], 'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'pulse', 'start_counter', 'time_data'], 'time': ['time_h', 'time_m', 'time_s']}


In [32]:
df_tdc

,channel,start_counter,high_voltage (V),pulse,time_data
0,1,8202,600.0,328.0,92186
1,3,8202,600.0,328.0,92507
2,2,8202,600.0,328.0,92510
3,0,8202,600.0,328.0,92847
4,2,13183,600.0,328.0,85018
...,...,...,...,...,...
59971104,0,9312,8000.0,1600.0,124720
59971105,3,9312,8000.0,1600.0,125864
59971106,2,9312,8000.0,1600.0,125796
59971107,1,14307,8000.0,1600.0,66948


In [33]:
channel = df_tdc['channel'].to_numpy()
start_counter = df_tdc['start_counter'].to_numpy()
time_data = df_tdc['time_data'].to_numpy()

In [91]:
def find_consecutive_sequences(arr, channel, time_data):
    result_4 = []
    result_3 = []
    result_2 = []
    result_1 = []
    result_other_odd = []
    result_other_even = []

    current_sequence = []
    current_values = []
    current_start = 0

    for i, value in enumerate(arr):
        if not current_sequence or current_sequence[-1] == value:
            current_sequence.append(value)
            current_values.append(value)
        else:
            length = len(current_sequence)
            if length == 4:
                result_4.append({
                    'channels': channel[current_start:  i],
                    'time_data': time_data[current_start:  i],
                    'values': current_values.copy(),
                    'indices': (current_start, i - 1),
                    'length': length
                    })
            elif length == 3:
                result_3.append({
                    'channels': channel[current_start:  i],
                    'time_data': time_data[current_start:  i],
                    'values': current_values.copy(),
                    'indices': (current_start, i - 1),
                    'length': length
                    })
            elif length == 2:
                result_2.append({
                    'channels': channel[current_start:  i],
                    'time_data': time_data[current_start:  i],
                    'values': current_values.copy(),
                    'indices': (current_start, i - 1),
                    'length': length
                    })
            elif length == 1:
                result_1.append({
                    'channels': channel[current_start:  i],
                    'time_data': time_data[current_start:  i],
                    'values': current_values.copy(),
                    'indices': (current_start, i - 1),
                    'length': length
                    })
            else:
                if length % 2 == 0:
                    result_other_even.append({
                        'channels': channel[current_start:  i],
                        'time_data': time_data[current_start:  i],
                        'values': current_values.copy(),
                        'indices': (current_start, i - 1),
                        'length': length
                        })
                else:
                    result_other_odd.append({
                        'channels': channel[current_start:  i],
                        'time_data': time_data[current_start:  i],
                        'values': current_values.copy(),
                        'indices': (current_start, i - 1),
                        'length': length
                        })

            current_sequence = [value]
            current_values = [value]
            current_start = i

    # Handle the last sequence
    length = len(current_sequence)
    if length == 4:
        result_4.append({
            'channels': channel[current_start:  len(arr)],
            'time_data': time_data[current_start:  len(arr)],
            'values': current_values.copy(),
            'indices': (current_start, len(arr) - 1),
            'length': length,
            })
    elif length == 3:
        result_3.append({
            'channels': channel[current_start:  len(arr)],
            'time_data': time_data[current_start:  len(arr)],
            'values': current_values.copy(),
            'indices': (current_start, len(arr) - 1),
            'length': length,
            })
    elif length == 2:
        result_2.append({
            'channels': channel[current_start:  len(arr)],
            'time_data': time_data[current_start:  len(arr)],
            'values': current_values.copy(),
            'indices': (current_start, len(arr) - 1),
            'length': length,
            })
    elif length == 1:
        result_1.append({
            'channels': channel[current_start:  len(arr)],
            'time_data': time_data[current_start:  len(arr)],
            'values': current_values.copy(),
            'indices': (current_start, len(arr) - 1),
            'length': length,
            })
    else:
        if length % 2 == 0:
            result_other_even.append({
                'channels': channel[current_start:  len(arr)],
                'time_data': time_data[current_start:  len(arr)],
                'values': current_values.copy(),
                'indices': (current_start, len(arr) - 1),
                'length': length,
            })
        else:
            result_other_odd.append({
                'channels': channel[current_start:  len(arr)],
                'time_data': time_data[current_start:  len(arr)],
                'values': current_values.copy(),
                'indices': (current_start, len(arr) - 1),
                'length': length,
            })

    lenght_result_4 = sum(item['length'] for item in result_4)
    lenght_result_3 = sum(item['length'] for item in result_3)
    lenght_result_2 = sum(item['length'] for item in result_2)
    lenght_result_1 = sum(item['length'] for item in result_1)
    lenght_result_other_odd = sum(item['length'] for item in result_other_odd)
    lenght_result_other_even = sum(item['length'] for item in result_other_even)

    print(f"Length of result_4: {lenght_result_4}, {lenght_result_4 / len(arr) * 100} %")
    print(f"Length of result_3: {lenght_result_3}, {lenght_result_3 / len(arr) * 100} %")
    print(f"Length of result_2: {lenght_result_2}, {lenght_result_2 / len(arr) * 100} %")
    print(f"Length of result_1: {lenght_result_1}, {lenght_result_1 / len(arr) * 100} %")
    print(f"Length of result_other_odd: {lenght_result_other_odd}, {lenght_result_other_odd / len(arr) * 100} %")
    print(f"Length of result_other_even: {lenght_result_other_even}, {lenght_result_other_even / len(arr) * 100} %")

    # Check the conditions
    total_length = (lenght_result_4 + lenght_result_3 + lenght_result_2 + lenght_result_1 + lenght_result_other_odd +
                    lenght_result_other_even)
    assert total_length == len(arr), "The total length of the sequences is not equal to the length of the array"
    print(f"Total length: {total_length}")

    return result_4, result_3, result_2, result_1, result_other_odd, result_other_even


result_4, result_3, result_2, result_1, result_other_odd, result_other_even = find_consecutive_sequences(start_counter,
                                                                                                         channel,
                                                                                                         time_data)

Length of result_4: 47481844, 79.17453052268884 %
Length of result_3: 6532554, 10.89283508163906 %
Length of result_2: 3311822, 5.522362442888959 %
Length of result_1: 1018779, 1.6987829923238538 %
Length of result_other_odd: 481056, 0.802146246786932 %
Length of result_other_even: 1145054, 1.909342713672345 %
Total length: 59971109


In [94]:
result_other_even[:5]

[{'channels': array([1, 3, 2, 0, 1, 2, 3, 0], dtype=uint32),
  'time_data': array([147378, 148557, 147653, 148831, 158951, 158805, 160231, 160095],
        dtype=uint32),
  'values': [17798, 17798, 17798, 17798, 17798, 17798, 17798, 17798],
  'indices': (2704, 2711),
  'length': 8},
 {'channels': array([1, 3, 1, 2, 0, 3, 0, 2], dtype=uint32),
  'time_data': array([60820, 60910, 61947, 60369, 61467, 61912, 61786, 62836],
        dtype=uint32),
  'values': [2904, 2904, 2904, 2904, 2904, 2904, 2904, 2904],
  'indices': (3686, 3693),
  'length': 8},
 {'channels': array([3, 1, 0, 2, 3, 1, 0, 2, 3, 1, 0, 2], dtype=uint32),
  'time_data': array([ 6051,  7183,  6859,  7990, 29820, 30914, 30651, 31749, 37901,
         38672, 38572, 39344], dtype=uint32),
  'values': [1965,
   1965,
   1965,
   1965,
   1965,
   1965,
   1965,
   1965,
   1965,
   1965,
   1965,
   1965],
  'indices': (4097, 4108),
  'length': 12},
 {'channels': array([2, 1, 3, 0, 2, 0], dtype=uint32),
  'time_data': array([8019

In [95]:
result_other_odd[:5]

[{'channels': array([3, 0, 2, 3, 0, 1, 2], dtype=uint32),
  'time_data': array([53891, 53439, 54712, 57388, 56940, 58626, 58189], dtype=uint32),
  'values': [11020, 11020, 11020, 11020, 11020, 11020, 11020],
  'indices': (318, 324),
  'length': 7},
 {'channels': array([3, 1, 0, 3, 1, 0, 2], dtype=uint32),
  'time_data': array([254688, 255886, 255354, 312525, 313715, 313184, 314373],
        dtype=uint32),
  'values': [7579, 7579, 7579, 7579, 7579, 7579, 7579],
  'indices': (940, 946),
  'length': 7},
 {'channels': array([3, 1, 0, 3, 1, 0, 2], dtype=uint32),
  'time_data': array([61424, 62422, 62451, 69833, 70810, 70838, 71814], dtype=uint32),
  'values': [17234, 17234, 17234, 17234, 17234, 17234, 17234],
  'indices': (1706, 1712),
  'length': 7},
 {'channels': array([3, 1, 3, 0, 2, 1, 0, 2, 3, 1, 3, 1, 0], dtype=uint32),
  'time_data': array([33299, 33464, 35348, 34747, 34886, 35736, 36816, 37205, 66378,
         66861, 68685, 69255, 70225], dtype=uint32),
  'values': [17106,
   17106,